In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('data_sets/train.csv')
df.head()

In [2]:
# selection of the model
model = LogisticRegression(multi_class='multinomial')

In [ ]:
# encode the categorical column to numerical values with dictionary (Important piece of code)
transform_dic = {'Sex': {"female":0, "male":1},
                'Embarked' : {'C':0, 'Q':1, 'S':2}}
tran_df = df.replace(transform_dic)
tran_df.head()